In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import eval_paths
import re

In [2]:
def Q_to_subscript(s):
    if 'GOODQ' in s:
        match = re.search(r'\d+', s)
        name = r'GOOD\textsubscript{' + match.group(0) + '}'
        return name
    else:
        return s

In [3]:
def forward(a):
    a = np.deg2rad(a)
    return np.rad2deg(np.log(np.abs(np.tan(a) + 1.0 / np.cos(a))))


def inverse(a):
    a = np.deg2rad(a)
    return np.rad2deg(np.arctan(np.sinh(a)))


In [4]:
def plot_hist(confs, color):
    ymax = len(confs)
    y_change = 0.2
    def forward(a):
        return (a <= y_change*ymax) * a + (a > y_change*ymax) * (y_change*ymax + 0.1*(a-y_change*ymax))
    def inverse(b):
        return (b <= y_change*ymax)*b + (b > y_change*ymax) * y_change*ymax + 1/0.1*(b-y_change*ymax)
    fig = plt.hist(confs, color=color, bins=50, align='mid', range=(0.1, 1.0))
    plt.ylim(ymin=0, ymax=1.0*ymax)
    plt.yscale('function', functions=(forward, inverse))
    plt.xlim(xmin=0.09, xmax=1.02)
    plt.xticks(np.arange(0.1, 1.01, 0.3))
    if 5000 < ymax < 20000:
        plt.yticks(np.concatenate([np.arange(0, ymax*y_change+1, 500), np.arange(4000, ymax-1000, 2000), [ymax]]))
    elif 20000 < ymax < 25000:
        plt.yticks(np.concatenate([np.arange(0, ymax*y_change+1, 1000), np.arange(8000, ymax-2000, 4000), [ymax]]))
    elif 25000 < ymax < 40000:
        plt.yticks(np.concatenate([np.arange(0, ymax*y_change+1, 1500), np.arange(12000, ymax-3000, 6000), [ymax]]))
    elif 100 < ymax < 1000:
        plt.yticks(np.concatenate([np.arange(0, ymax*y_change+1, 20), np.arange(100, ymax-30, 100), [ymax]]))
    plt.axvline(confs.mean(), color='k', linestyle="-", dashes=(1, 3), linewidth=2)
    #plt.axvline(pred_in_confidences.mean(), color='w', linestyle="-", dashes=(2, 6), linewidth=2)
    plt.axhline(y_change*ymax, color='#DDDDDD', linestyle="--", linewidth=1)
    #dotted_line2 = lines.Line2D([], [], linewidth=5, linestyle="-", dashes=(5, 4), color='black')
    #save_path = model_folder + f'/sample_images/{short_name}{in_name}_in_pred_confidences_hist'
    #plt.savefig(save_path, bbox_inches='tight')
    #plt.show()
    #plt.close()
    return fig
    #break

In [5]:
eval_log_cifar10_filedest = 'evals/2020-10-14_16:34:38.log'
in_name_c10 = 'CIFAR10'
out_names_c10 = ['CIFAR100', 'SVHN', 'LSUN', 'Noise_Dataset_noise_uniform']
model_list_c10 = ['OE', 'ACET', 'GOODQ40', 'GOODQ80', 'GOODQ90', 'GOODQ100']

In [6]:
eval_log_svhn_filedest = 'evals/2020-10-14_18:14:40.log'
in_name_svhn = 'SVHN'
out_names_svhn = ['CIFAR100', 'CIFAR10', 'LSUN', 'Noise_Dataset_noise_uniform']
model_list_svhn = ['OE', 'ACET', 'GOODQ40', 'GOODQ80', 'GOODQ90', 'GOODQ100']

In [7]:
eval_log_mnist_filedest = 'evals/2020-10-15_15:13:11.log'
in_name_mnist = 'MNIST'
out_names_mnist = ['FashionMNIST', 'EMNIST', 'CIFAR10', 'Noise_Dataset_noise_uniform']
model_list_mnist = ['Plain', 'OE', 'ACET', 'GOODQ40', 'GOODQ80', 'GOODQ90', 'GOODQ100']

In [8]:
def make_histograms(eval_log_filedest, in_name, out_names, model_list):
    eval_log = open(eval_log_filedest, 'r')
    model_rows = {}
    for l in list(eval_log):
        l = l[:-1]
        if l[:len('Evaluation outputs saved in')] == 'Evaluation outputs saved in':
            shortname = eval_paths.get_shortname(l) 
            if shortname not in model_list:
                continue
            values_folder = l.split(' ')[-1] + '/values/'
            histsave_folder =  'evals/plots/histograms_final/'
            os.makedirs(histsave_folder, exist_ok=True)
            if shortname != 'Plain':
                pred_in_confidences = np.loadtxt(values_folder + f'{shortname}_{in_name}_pred_in_confidences.txt')
            else:
                pred_in_confidences = np.loadtxt(values_folder + f'plain_{in_name}_pred_in_confidences.txt')
            fig = plot_hist(pred_in_confidences, '#00CC44')
            filename = f'{in_name}-{shortname}_{in_name}_pred_confidences_hist.png'
            save_path = histsave_folder + filename
            plt.savefig(save_path, bbox_inches='tight')
            #plt.show()
            plt.close()
            print(rf'{Q_to_subscript(shortname)}')
            print(rf' &  \includegraphics[width=1.0\linewidth]{{images/histograms_final/{filename}}}')
            for n in out_names:
                #print(n)
                #pred_out_confidences = np.loadtxt(values_folder + f'{shortname}_{n}_pred_out_confidences.txt')
                #fig = plot_hist(pred_out_confidences, '#4400DD')
                #plt.show()
                #plt.close()
                if shortname != 'Plain':
                    ub_out_confidences = np.loadtxt(values_folder + f'{shortname}{n}_ub_el_out_confidences.txt')
                else:
                    ub_out_confidences = np.loadtxt(values_folder + f'plain{n}_ub_el_out_confidences.txt')
                fig = plot_hist(ub_out_confidences, '#4400DD')
                filename = f'{in_name}-{shortname}_{n}_ub_confidences_hist.png'
                save_path = histsave_folder + filename
                plt.savefig(save_path, bbox_inches='tight')
                #plt.show()
                plt.close()
                print(rf' &  \includegraphics[width=1.0\linewidth]{{images/histograms_final/{filename}}}')
            print('\\\\')

In [9]:
make_histograms(eval_log_mnist_filedest, in_name_mnist, out_names_mnist, model_list_mnist)

Plain
 &  \includegraphics[width=1.0\linewidth]{images/histograms_final/MNIST-Plain_MNIST_pred_confidences_hist.png}
 &  \includegraphics[width=1.0\linewidth]{images/histograms_final/MNIST-Plain_FashionMNIST_ub_confidences_hist.png}
 &  \includegraphics[width=1.0\linewidth]{images/histograms_final/MNIST-Plain_EMNIST_ub_confidences_hist.png}
 &  \includegraphics[width=1.0\linewidth]{images/histograms_final/MNIST-Plain_CIFAR10_ub_confidences_hist.png}
 &  \includegraphics[width=1.0\linewidth]{images/histograms_final/MNIST-Plain_Noise_Dataset_noise_uniform_ub_confidences_hist.png}
\\
OE
 &  \includegraphics[width=1.0\linewidth]{images/histograms_final/MNIST-OE_MNIST_pred_confidences_hist.png}
 &  \includegraphics[width=1.0\linewidth]{images/histograms_final/MNIST-OE_FashionMNIST_ub_confidences_hist.png}
 &  \includegraphics[width=1.0\linewidth]{images/histograms_final/MNIST-OE_EMNIST_ub_confidences_hist.png}
 &  \includegraphics[width=1.0\linewidth]{images/histograms_final/MNIST-OE_CIFAR1

In [10]:
def latex_len5(s):
    if len(s) < 5:
        return '\\phantom{' + (5-len(s))*'0' + '}' + s
    else:
        return s

In [11]:
def make_conf_table(eval_log_filedest, in_name, out_names, model_list):
    eval_log = open(eval_log_filedest, 'r')
    model_rows = {}
    for l in list(eval_log):
        l = l[:-1]
        if l[:len('Evaluation outputs saved in')] == 'Evaluation outputs saved in':
            shortname = eval_paths.get_shortname(l) 
            if shortname not in model_list:
                continue
            values_folder = l.split(' ')[-1] + '/values/'
            if shortname != 'Plain':
                pred_in_confidences = np.loadtxt(values_folder + f'{shortname}_{in_name}_pred_in_confidences.txt')
            else:
                pred_in_confidences = np.loadtxt(values_folder + f'plain_{in_name}_pred_in_confidences.txt')
            model_rows[shortname] = rf'{Q_to_subscript(shortname)}'
            model_rows[shortname] += f'\n & {100*pred_in_confidences.mean():.1f}'
            for n in out_names:
                if shortname != 'Plain':
                    ub_out_confidences = np.loadtxt(values_folder + f'{shortname}{n}_ub_el_out_confidences.txt')
                    pred_out_confidences = np.loadtxt(values_folder + f'{shortname}{n}_pred_out_confidences.txt')
                else:
                    ub_out_confidences = np.loadtxt(values_folder + f'plain{n}_ub_el_out_confidences.txt')
                    pred_out_confidences = np.loadtxt(values_folder + f'plain{n}_pred_out_confidences.txt')
                model_rows[shortname] += f'\n & ' + f'{100*pred_out_confidences.mean():.1f}' + '\ / ' + latex_len5(f'{100*ub_out_confidences.mean():.1f}')
            model_rows[shortname] += '\\\\'
    for model in model_list:
        print(model_rows[model])

In [12]:
model_list_conf = ['Plain', 'CEDA', 'OE', 'ACET', 'GOODQ0', 'GOODQ20', 'GOODQ40', 'GOODQ60', 'GOODQ80', 'GOODQ90', 'GOODQ95', 'GOODQ100']

In [13]:
out_names_mnist

['FashionMNIST', 'EMNIST', 'CIFAR10', 'Noise_Dataset_noise_uniform']

In [14]:
make_conf_table(eval_log_mnist_filedest, in_name_mnist, out_names_mnist, model_list_conf)

Plain
 & 99.7
 & 79.2\ / 100.0
 & 91.5\ / 100.0
 & 77.2\ / 100.0
 & 79.6\ / 100.0\\
CEDA
 & 99.7
 & 22.0\ / 100.0
 & 88.3\ / 100.0
 & 10.0\ / 100.0
 & 10.0\ / 100.0\\
OE
 & 99.7
 & 25.4\ / 100.0
 & 87.9\ / 100.0
 & 10.1\ / 100.0
 & 10.0\ / 100.0\\
ACET
 & 99.6
 & 12.3\ / 100.0
 & 75.0\ / 100.0
 & 10.0\ / 100.0
 & 10.0\ / 100.0\\
GOOD\textsubscript{0}
 & 99.7
 & 20.6\ / 100.0
 & 87.9\ / 100.0
 & 10.0\ / 100.0
 & 10.0\ / 100.0\\
GOOD\textsubscript{20}
 & 99.5
 & 19.4\ / \phantom{0}93.0
 & 70.2\ / 100.0
 & 10.0\ / \phantom{0}76.6
 & 10.0\ / \phantom{0}10.0\\
GOOD\textsubscript{40}
 & 99.3
 & 17.7\ / \phantom{0}76.8
 & 58.2\ / 100.0
 & 10.0\ / \phantom{0}43.7
 & 10.0\ / \phantom{0}10.0\\
GOOD\textsubscript{60}
 & 99.2
 & 15.8\ / \phantom{0}66.0
 & 51.7\ / 100.0
 & 10.0\ / \phantom{0}24.8
 & 10.0\ / \phantom{0}10.0\\
GOOD\textsubscript{80}
 & 99.0
 & 16.3\ / \phantom{0}55.1
 & 40.7\ / \phantom{0}98.6
 & 10.0\ / \phantom{0}15.7
 & 10.0\ / \phantom{0}10.0\\
GOOD\textsubscript{90}
 & 98.8
 & 1